<a href="https://colab.research.google.com/github/timgluz/colab_notebooks/blob/master/HumanPoseEstimation_project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading data

Tutorial and hints about the dataset: 

https://www.tensorflow.org/datasets/catalog/svhn_cropped


In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load(
    name = "svhn_cropped",
    with_info= True,
    batch_size = 32,
    as_supervised = True)


In [ ]:
# Preparing data
print(f"Training size: {info.splits['train'].num_examples}")
print(f"Test size: {info.splits['test'].num_examples}")

In [ ]:
svhn_train = dataset['train'].prefetch(1)
svhn_test = dataset['test'].prefetch(1)


In [ ]:
t1 = dataset['train']

# Building a model

Let’s now implement the architecture with 3 convolutional layers. Check the dimensions of the feature maps after each layer.

**Notice** how the dimensions change when you change padding or stride. The configuration below should give you an accuracy of about 88% on the SVHN dataset. Note: Make sure you use the “2d”-versions of the units.

```
 1. convolution, kernel_size=5, channels=6, stride=1, padding=2
 2. batch-normalization
 3. ReLU
 4. Max-pool, kernel_size=2, stride=2

 5. convolution, kernel_size=3, channels=12, stride=1, padding=1
 6. batch-normalization
 7. ReLU
 8. Max-pool, kernel_size=2, stride=2

 9. convolution, kernel_size=3, channels=24, stride=1, padding=1
 10. batch-normalization
 11. ReLU
 12. Max-pool, kernel_size=2, stride=2

 13. fully connected layer, output_size=10
```

In [ ]:

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, ReLU, Dense
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

model  = tf.keras.models.Sequential([
  Rescaling(1.0/255, input_shape = [32, 32, 3]),                            
  Conv2D(16, kernel_size = 5, strides = (1,1),
         input_shape= [32, 32, 3], padding = "valid"),
  BatchNormalization(),
  ReLU(),

  # 2nd layer
  Conv2D(32, kernel_size = 3, strides = (1, 1), padding = "valid"),
  BatchNormalization(),
  ReLU(),
  MaxPool2D(pool_size = (2, 2), strides = (2, 2)),

  # 3rd layer
  Conv2D(64, kernel_size = 3, strides = (1, 1), padding = "valid"),
  BatchNormalization(),
  ReLU(),
  MaxPool2D(pool_size = (2, 2), strides = (2, 2)),

  # final layer
  Dense(10)
])

model.build()
print(model.summary())


In [ ]:


model.compile(
  loss = "categorical_crossentropy",
  optimizer = tf.keras.optimizers.Adam(lr = 0.001),
  metrics = ["accuracy"]
)

model.fit(svhn_train, epochs = 5, batch_size = 32)